Import Packages

In [1]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools

Files for Gephi

In [7]:
df = pd.read_csv(r'./dataset.csv')

df2 = df[["track_id","writers","track_name","track_pos"]]
df2_dict = df2.to_dict(orient='records')

d = []

for f in df2_dict:
    d.extend(f["writers"].split("-"))

set_d = set([a.strip() for a in d])
dict_vett_writers = []
new_dict_vect_writers = []
support = []
support2 = ""

for writer in set_d:
    for d in df2_dict:
        if writer in [a.strip() for a in d["writers"].split("-")]:
            support.append({"name":d["track_name"],"pos":d["track_pos"]})
            support2 += d["track_name"]+" - "+str(d["track_pos"])+","
    dict_vett_writers.append({"writer":writer,"tracks":support})
    new_dict_vect_writers.append({"writer":writer,"tracks":support2[:-1].split(",")})
    support = []
list_edges = []
l_e_new = []
list_weights = []

for dic in dict_vett_writers:
    for dic2 in dict_vett_writers:
        for song in dic["tracks"]:
            if song in dic2["tracks"] and dic["writer"] != dic2["writer"]:
                if {"source":dic["writer"],"target":dic2["writer"]} in list_edges:
                    index = list_edges.index({"source":dic["writer"],"target":dic2["writer"]})
                    list_weights[index] += (10-song["pos"])
                else:
                    common_tracks =[]
                    list_edges.append({"source":dic["writer"],"target":dic2["writer"]})
                    for t1 in dic["tracks"]:
                        if t1 in dic2["tracks"]:
                            common_tracks.append(t1)
                    l_e_new.append({"source":dic["writer"],"target":dic2["writer"],"tracks":common_tracks})
                    list_weights.append(1 + (10-song["pos"]))

for weight in list_edges:
    a = weight["source"]
    b = weight["target"]
    index = list_edges.index(weight)
    try:
        list_edges.remove({"source":b,"target":a})
        list_weights.remove(list_weights[index])
    except:
        pass

lista =list(set([str(a["writer"]).strip() for a in dict_vett_writers]))
print(True if ("M. Vicino") in lista else False)
'''for w in lista:
    if w not in [b["source"] for b in list_edges]:
        lista.remove(w)'''

True


'for w in lista:\n    if w not in [b["source"] for b in list_edges]:\n        lista.remove(w)'

Creation of the files

In [8]:
writers = open("artisti.csv","w")
writers.write("Id,Label\n")
writers.write('\n'.join([str(a) + "," + str(a) for a in lista]))#('\n'.join(lista))
writers.close()

writersTracks = open("artisti_e_canzoni.csv","w")
writersTracks.write("Writer,Songs\n")
writersTracks.write('\n'.join([str(a["writer"]) + "," + str(a["tracks"]) for a in dict_vett_writers]))
writersTracks.close()

outfile = open("edges_list.csv","w")
outfile.write("Source,Target,Weight\n")
outfile.write('\n'.join([str(list_edges[i]["source"]) + "," + str(list_edges[i]["target"]) +","+ str(list_weights[i]) for i in range( len(list_edges))]))
outfile.close()

Create a Graph

In [22]:
G = nx.Graph()
G.add_nodes_from(lista)
ebunch = [(list_edges[i]["source"],list_edges[i]["target"],{"weight":list_weights[i]}) for i in range( len(list_edges))]
G.add_edges_from(ebunch)

gMogol = nx.Graph()
gMogol.add_nodes_from(lista)
ebunchMogol = [(list_edges[i]["source"],list_edges[i]["target"],{"weight":list_weights[i]}) for i in range( len(list_edges)) if list_edges[i]["source"] == "Mogol" or list_edges[i]["target"] == "Mogol"]
gMogol.add_edges_from(ebunchMogol)
gMogol.remove_nodes_from(list(nx.isolates(gMogol)))

#degree centrality
print("degree centrality")
deg_centrality = nx.degree_centrality(G)
#nx.write_gexf(deg_centrality, "deg_cent.gexf")
print(deg_centrality,'\n')


#degree distribution
print("degree distribution")
print(nx.degree_histogram(G),'\n') #find probability

#pagerank
print("pagerank")
print(nx.pagerank(G, weight="weight"),'\n')

#average clustering coefficient
print("average clustering coefficient")
print(nx.average_clustering(G,weight="weight"),'\n')



#k-cliques
cliqueMatrix = []
cliqueMatrix.append([])
cliqueMatrix.append([])
for k in range(2,8):
    i = 0
    cliqueMatrix.append([])
    for clique in nx.find_cliques(G):
        if len(clique) == k:
            i += 1
            cliqueMatrix[k].append(clique)
        '''elif len(clique) > k:
            i += len(list(itertools.combinations(clique, k)))'''
    print(k,"-clique: ", i)
print('\n')

#clique edges experimental
def travel_list(l,k):
    for el in l:
        a = el.get(k,None)
        if a == None:
            continue
        else:
            return a

cliqueEdges= []
target = [{str(a["source"] + a["target"]):{"s":a["source"],"d":a["target"],"tracks":','.join([f["name"] + " - " + str(f["pos"]) for f in a["tracks"]])}} for a in l_e_new]
#print("target - ", target)
for i in range(len(cliqueMatrix[6])):
    cliqueEdges.append([])
    for node1 in cliqueMatrix[6][i]:
        s = []
        for node2 in cliqueMatrix[6][i]:
            t = travel_list(target,str(node1 + node2))
            if not t in s and t is not None:
                s.append(t)
        cliqueEdges.append(list(s))

result = [c for c in cliqueEdges ]#if len(c)>1]
#print("RES - ", result)

#############################
#find how many songs in every clique
cliqueSongs = []
for i in range(len(cliqueMatrix[6])):
    songs = []
    append_this = []
    for artist in cliqueMatrix[6][i]:
        for d in dict_vett_writers:
            if d["writer"] == artist:
                for song in d["tracks"]:
                    songs.append(song["name"] +" - " +str(song["pos"]))
        songs_set = set(songs)
        append_this = [[s,0] for s in songs_set]
        for song in songs:
            for j in range(len(append_this)):
                if song == append_this[j][0]:
                    append_this[j][1] += 1 #(song,append_this[j][1]+1)
        append_this = [a[0] for a in append_this if a[1] > 1]

    cliqueSongs.append("Clique number " + str(i) + " - " + str(append_this))
print("clique songs - ", cliqueSongs)

#############################

#closeness centrality
print("closeness centrality")
print(nx.closeness_centrality(G),'\n')

#betweenness centrality
print("betweenness centrality")
print(nx.betweenness_centrality(G,k=len(G.nodes()), normalized=True, weight="weight"),'\n')



#Degree Centrality, Degree Distibution, PageRank, Local Clustering Coefficient, N-cliques, Closeness, Betweenness

degree centrality
{'Boss Doms': 0.0040650406504065045, 'M. Monticelli': 0.005081300813008131, 'F. Di Gesù': 0.005081300813008131, 'M. De Paolis': 0.0010162601626016261, 'C. Chiodo': 0.003048780487804878, 'M. Di Franco': 0.0020325203252032522, 'Don Backy': 0.0020325203252032522, "B. D'Onghia": 0.0020325203252032522, 'A. Canini': 0.003048780487804878, 'R. Sanna': 0.0010162601626016261, 'E. Caterini': 0.0020325203252032522, 'E. Capotosti': 0.006097560975609756, 'F. Neviani': 0.0040650406504065045, 'S. Melloni': 0.0020325203252032522, 'F. Monti Arduini': 0.0010162601626016261, 'C. Consoli': 0.0020325203252032522, 'S. Sichel': 0.0020325203252032522, 'M. Grilli': 0.0020325203252032522, 'D. Fossati': 0.0010162601626016261, 'A. Di Martino': 0.0010162601626016261, 'G. Zagni': 0.0020325203252032522, 'E. Giudizi': 0.0020325203252032522, 'A. Anastasio': 0.0020325203252032522, 'Achille Lauro': 0.0040650406504065045, 'M. Calabrese': 0.008130081300813009, 'Daniele Silvestri': 0.0, 'P. Limiti': 0.0040

In [23]:
newfile = open("ce.txt","w")
newfile.write('\n'.join([str(b["s"]) + " | " + str(b["d"]) + " | " + str(b["tracks"]) for a in result for b in a])) 
newfile.close()

newfile2 = open("deg_centrality.csv","w")
newfile2.write("Writer,Label,Degree Centrality\n")
newfile2.write('\n'.join([str(a) + ","+ str(a) +"," + str(deg_centrality[a]) for a in deg_centrality]))
newfile2.close()

nx.write_gexf(gMogol, "mogol.gexf")
